## HuggingFace Transformers를 활용한 문장 분류 모델 학습
본 노트북에서는 klue/roberta-base 모델을 KLUE 내 NLI 데이터셋을 활용하여 모델을 훈련하는 예제를 다루게 됩니다.

학습을 통해 얻어질 klue-roberta-base-nli 모델은 입력된 두 문장의 추론 관계를 예측하는데 사용할 수 있게 됩니다.

학습 과정 이후에는 간단한 예제 코드를 통해 모델이 어떻게 활용되는지도 함께 알아보도록 할 것입니다.

모든 소스 코드는 huggingface-notebooks를 참고하였습니다.

먼저, 노트북을 실행하는데 필요한 라이브러리를 설치합니다. 모델 훈련을 위해서는 transformers가, 학습 데이터셋 로드를 위해서는 datasets 라이브러리의 설치가 필요합니다. 그 외 모델 성능 검증을 위해 scipy, scikit-learn을 추가로 설치해주도록 합니다.

In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers datasets evaluate

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [38]:
import random
import logging
from IPython.display import display, HTML
import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric,ClassLabel, Sequence
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [39]:
model_ckpt = "klue/roberta-base"
batch_size = 32
task = "nli"

In [41]:
datasets = load_dataset("klue",task)

Found cached dataset klue (C:/Users/clee/.cache/huggingface/datasets/klue/nli/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

In [53]:
datasets.set_format(type="pandas")
df = datasets["train"][:]
def label_int2str(row):
    return datasets["train"].features["label"].int2str(row)
df["label_name"] = df["label"].apply(label_int2str)
df.head(30)

,guid,source,premise,hypothesis,label,label_name
0,klue-nli-v1_train_00000,NSMC,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 진심 최고로 멋지다.,0,entailment
1,klue-nli-v1_train_00001,NSMC,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 잤다.,2,contradiction
2,klue-nli-v1_train_00002,NSMC,100분간 잘껄 그래도 소닉붐땜에 2점준다,소닉붐이 정말 멋있었다.,1,neutral
3,klue-nli-v1_train_00003,NSMC,100분간 잘껄 그래도 소닉붐땜에 2점준다,100분간 자는게 더 나았을 것 같다.,1,neutral
4,klue-nli-v1_train_00004,airbnb,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 근처에서 즐길거리 찾기는 어렵습니다.,2,contradiction
5,klue-nli-v1_train_00005,airbnb,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 주변에 젊은이들이 즐길거리가 많습니다.,1,neutral
6,klue-nli-v1_train_00006,airbnb,101빌딩 근처에 나름 즐길거리가 많습니다.,101빌딩 부근에서는 여러가지를 즐길수 있습니다.,0,entailment
7,klue-nli-v1_train_00007,wikipedia,10년 만에 찾는 피터를 웬디는 따뜻하게 맞이하고 피터는 성공리에 연설을 마치는데 ...,웬디는 피터를 차갑게 맞이했다.,2,contradiction
8,klue-nli-v1_train_00008,wikipedia,10년 만에 찾는 피터를 웬디는 따뜻하게 맞이하고 피터는 성공리에 연설을 마치는데 ...,잭과 매기는 피터 배닝의 동생들이다.,1,neutral
9,klue-nli-v1_train_00009,wikipedia,10년 만에 찾는 피터를 웬디는 따뜻하게 맞이하고 피터는 성공리에 연설을 마치는데 ...,"피터 배닝, 잭, 매기는 남매사이다.",0,entailment


In [ ]:
datasets.reset_format()

In [32]:
import evaluate
metric = evaluate.load("glue","mnli")

In [33]:
fake_preds = np.random.randint(0, 2, size=(10,))
fake_labels = np.random.randint(0, 2, size=(10,))
fake_preds, fake_labels

(array([1, 0, 1, 1, 1, 1, 0, 1, 0, 0]), array([0, 0, 1, 1, 1, 0, 0, 1, 0, 1]))

In [35]:
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.7}

In [43]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
inputs = tokenizer("힛걸 진심 최고로 멋지다.", "힛걸 진심 최고다 그 어떤 히어로보다 멋지다")

In [44]:
tokenizer.decode(inputs["input_ids"])

'[CLS] [UNK] 진심 최고로 멋지다. [SEP] [UNK] 진심 최고다 그 어떤 히어로보다 멋지다 [SEP]'

In [45]:
sentence1_key, sentence2_key = ("premise", "hypothesis")

In [46]:
def tokenize_function(examples):
    return tokenizer(examples[sentence1_key],examples[sentence2_key],truncation=True,return_token_type_ids=False)
datasets_encoded = datasets.map(tokenize_function,batched=True)

Map:   0%|          | 0/24998 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
id2label = {0: "entailment",1: "neutral",2: "contradiction"}
label2id = {"entailment": 0,"neutral": 1, "contradiction": 2}

In [47]:
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt,num_labels=num_labels,id2label=id2label,label2id=label2id)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You

In [48]:
import evaluate
import numpy as np
def compute_metrics(eval_pred):
    # klue/nli uses the same metric as that of glue/mnli
    metric = evaluate.load("glue","mnli")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions,references=labels)

In [51]:
metric_name = "accuracy"
batch_size = 32
args = TrainingArguments(
    "klue_nli_roberta_base_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_gpu_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [52]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=datasets_encoded["train"],
    eval_dataset=datasets_encoded["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/chunwoolee0/klue_nli_roberta_base_model into local empty directory.


In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()